In [45]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [46]:
train_ds = tf.keras.utils.image_dataset_from_directory("./data/akshar",
                                                       subset = "training",
                                                       validation_split=0.2,
                                                       image_size = (224, 224),
                                                      seed = 777)

val_ds = tf.keras.utils.image_dataset_from_directory("./data/akshar",
                                                         validation_split = 0.2,
                                                     image_size = (224, 224),
                                                     subset="validation",
  seed=777)

Found 3369 files belonging to 34 classes.
Using 2696 files for training.
Found 3369 files belonging to 34 classes.
Using 673 files for validation.


In [47]:
num_classes = len(train_ds.class_names)
print(train_ds.class_names)

['ALA', 'ANA', 'B', 'BHA', 'CH', 'CHH', 'D', 'DA', 'DH', 'DHA', 'F', 'G', 'GH', 'GNA', 'H', 'J', 'JH', 'K', 'KH', 'KSH', 'L', 'M', 'N', 'P', 'R', 'S', 'SH', 'SHH', 'T', 'TA', 'TH', 'THA', 'V', 'Y']


In [39]:


AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)


normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
normalized_val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))



Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [40]:
import tensorflow_hub as hub
mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor_model = mobilenet_v2
feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224, 224, 3),
    trainable=False)

In [41]:

model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(num_classes)
])

model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_5 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dense_5 (Dense)             (None, 34)                43554     
                                                                 
Total params: 2,301,538
Trainable params: 43,554
Non-trainable params: 2,257,984
_________________________________________________________________


In [42]:
import datetime 
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1)

In [43]:
NUM_EPOCHS = 10

history = model.fit(train_ds,
                    validation_data = val_ds,
                    epochs=NUM_EPOCHS,
                    callbacks=tensorboard_callback)

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
85/85 [==============================] - ETA: 0s - loss: 2.5284 - acc: 0.3661WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f991fd2d0e0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
85/85 [==============================] - 52s 553ms/step - loss: 2.5284 - acc: 0.3661 - val_loss: 1.6418 - val_acc: 0.6553
Epoch 2/10
85/85 [==============================] - 47s 559ms/step - loss: 1.2423 - acc: 0.7726 - val_loss: 0.9926 - val_acc: 0.8351
Epoch 3/10
85/85 [==============================] - 46s 539ms/step - loss: 0.7773 - acc: 0.8999 - val_loss: 0.6977 - val_acc: 0.9138
Epoch 4/10
85/85 [==============================] - 46s 541ms/step - loss: 0.5531 - acc: 0.9421 - val_loss: 0.5458 - val_acc: 0.9257
Epoch 5/10
85/85 [==============================] - 48s 562ms/step - loss: 0.4250 - acc: 0.9592 - val_loss: 0.4250 - val_acc: 0.9525
Epoch 6/10
85/85 [=================